# 🧪 NSAI Experiment Notebook

**Neurosymbolic Runner Selection — A/B Comparison & Analysis**

| Version | Date | Author |
|---------|------|--------|
| 0.3.0 | 2026-02-06 | Wolfram Laube |

This notebook runs reproducible experiments comparing three runner selection strategies:

1. **Rule-Based** — always pick the first feasible runner (static baseline)
2. **Pure MAB** — UCB1 over all runners, no constraint filtering
3. **NSAI** — CSP filter → UCB1 (our neurosymbolic approach)

Metrics: cumulative reward, regret, convergence speed, selection distribution.

In [ ]:
import sys, os
import math, random, time
import json
from collections import Counter, defaultdict
from typing import Dict, List, Tuple

# Ensure nsai is importable
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
if 'google.colab' in sys.modules:
    sys.path.insert(0, '/content')

from nsai import NeurosymbolicBandit, __version__
from nsai.ontology import create_blauweiss_ontology

print(f'NSAI v{__version__}')
print(f'Runners: {list(create_blauweiss_ontology().runners.keys())}')

## 1. Ground Truth Definition

We simulate job execution with known per-runner success probabilities and durations.
This lets us compute **optimal reward** and therefore **regret**.

In [ ]:
# Ground truth: success probability per runner per job type
GROUND_TRUTH = {
    'docker-any': {
        'gitlab-runner-nordic':     {'p_success': 0.96, 'avg_duration': 18.0, 'cost': 0.01},
        'Mac Docker Runner':        {'p_success': 0.92, 'avg_duration': 25.0, 'cost': 0.00},
        'Mac2 Docker Runner':       {'p_success': 0.85, 'avg_duration': 35.0, 'cost': 0.00},
        'Linux Yoga Docker Runner': {'p_success': 0.95, 'avg_duration': 15.0, 'cost': 0.00},
    },
    'gcp': {
        'gitlab-runner-nordic':     {'p_success': 0.96, 'avg_duration': 18.0, 'cost': 0.01},
        'Mac Docker Runner':        {'p_success': 0.00, 'avg_duration': 999,  'cost': 0.00},
        'Mac2 Docker Runner':       {'p_success': 0.00, 'avg_duration': 999,  'cost': 0.00},
        'Linux Yoga Docker Runner': {'p_success': 0.00, 'avg_duration': 999,  'cost': 0.00},
    },
}

def simulate_job(runner: str, job_type: str, rng: random.Random) -> Tuple[bool, float]:
    """Simulate a job execution. Returns (success, duration_seconds)."""
    profile = GROUND_TRUTH[job_type].get(runner, {'p_success': 0.0, 'avg_duration': 60.0})
    success = rng.random() < profile['p_success']
    duration = max(5.0, rng.gauss(profile['avg_duration'], profile['avg_duration'] * 0.2))
    if not success:
        duration = max(duration, rng.gauss(60.0, 10.0))  # failures take longer
    return success, duration

def compute_reward(success: bool, duration: float, cost_per_min: float = 0.0) -> float:
    """Reward function matching NSAI/MAB."""
    if not success:
        return 0.0
    dur_min = duration / 60.0
    cost_penalty = cost_per_min * dur_min
    return 1.0 / (dur_min + cost_penalty + 0.1)

# Optimal runner for docker-any (highest expected reward)
for runner, prof in GROUND_TRUTH['docker-any'].items():
    expected = prof['p_success'] * (1.0 / (prof['avg_duration']/60 + prof['cost'] * prof['avg_duration']/60 + 0.1))
    print(f'{runner:30} E[reward] = {expected:.3f}')

## 2. Strategy Implementations

In [ ]:
class RuleBasedStrategy:
    """Always picks the first online runner (static baseline)."""
    def __init__(self):
        self.name = 'Rule-Based'
        self._default = 'gitlab-runner-nordic'
    
    def select(self, job_type: str) -> str:
        return self._default
    
    def update(self, runner, success, duration, cost=0.0):
        pass  # no learning


class PureMABStrategy:
    """UCB1 over ALL runners — no symbolic filtering."""
    def __init__(self, runners: list, c: float = 2.0):
        self.name = 'Pure MAB'
        self.c = c
        self._stats = {r: {'pulls': 0, 'total_reward': 0.0} for r in runners}
        self._total = 0
    
    def select(self, job_type: str) -> str:
        # Exploration: try each at least once
        for r, s in self._stats.items():
            if s['pulls'] == 0:
                return r
        # UCB1
        best, best_ucb = None, -1
        for r, s in self._stats.items():
            mean = s['total_reward'] / s['pulls']
            explore = self.c * math.sqrt(math.log(self._total + 1) / s['pulls'])
            ucb = mean + explore
            if ucb > best_ucb:
                best, best_ucb = r, ucb
        return best
    
    def update(self, runner, success, duration, cost=0.0):
        reward = compute_reward(success, duration, cost)
        self._stats[runner]['pulls'] += 1
        self._stats[runner]['total_reward'] += reward
        self._total += 1


class NSAIStrategy:
    """Full NSAI: CSP filter → UCB1."""
    def __init__(self):
        self.name = 'NSAI'
        self.nsai = NeurosymbolicBandit.create_default()
    
    def select(self, job_type: str) -> str:
        tags = [job_type] if job_type != 'docker-any' else ['docker-any']
        runner, _ = self.nsai.select_runner({'tags': tags})
        return runner or 'gitlab-runner-nordic'
    
    def update(self, runner, success, duration, cost=0.0):
        self.nsai.update(runner, success, duration, cost)

print('Strategies defined ✓')

## 3. Run Experiment

In [ ]:
def run_experiment(n_rounds=200, seed=42, job_type='docker-any'):
    """Run A/B experiment and return per-round results."""
    rng = random.Random(seed)
    runners = list(GROUND_TRUTH[job_type].keys())
    
    strategies = [
        RuleBasedStrategy(),
        PureMABStrategy(runners),
        NSAIStrategy(),
    ]
    
    # Compute optimal expected reward for regret
    best_expected = max(
        p['p_success'] * compute_reward(True, p['avg_duration'], p['cost'])
        for p in GROUND_TRUTH[job_type].values()
        if p['p_success'] > 0
    )
    
    results = {s.name: {
        'rewards': [], 'cum_reward': [], 'regret': [],
        'cum_regret': [], 'selections': []
    } for s in strategies}
    
    for t in range(n_rounds):
        for strategy in strategies:
            r = results[strategy.name]
            
            runner = strategy.select(job_type)
            success, duration = simulate_job(runner, job_type, rng)
            cost = GROUND_TRUTH[job_type].get(runner, {}).get('cost', 0.0)
            reward = compute_reward(success, duration, cost)
            
            strategy.update(runner, success, duration, cost)
            
            r['rewards'].append(reward)
            r['cum_reward'].append(sum(r['rewards']))
            r['regret'].append(best_expected - reward)
            r['cum_regret'].append(sum(r['regret']))
            r['selections'].append(runner)
    
    return results

results = run_experiment(n_rounds=300, seed=42)
print(f'Experiment complete: 300 rounds × 3 strategies')

## 4. Results: Cumulative Reward

In [ ]:
# Text-based results (works without matplotlib)
print('=== Cumulative Reward (after 300 rounds) ===\n')
for name, r in sorted(results.items(), key=lambda x: -x[1]['cum_reward'][-1]):
    total = r['cum_reward'][-1]
    avg = total / len(r['rewards'])
    bar = '█' * int(total / 5)
    print(f'{name:15} {total:7.1f}  (avg {avg:.3f})  {bar}')

print('\n=== Cumulative Regret (lower is better) ===\n')
for name, r in sorted(results.items(), key=lambda x: x[1]['cum_regret'][-1]):
    total = r['cum_regret'][-1]
    bar = '░' * int(total / 2)
    print(f'{name:15} {total:7.1f}  {bar}')

## 5. Results: Selection Distribution

In [ ]:
print('=== Runner Selection Distribution ===\n')
runners_short = {
    'gitlab-runner-nordic': 'nordic',
    'Mac Docker Runner': 'mac',
    'Mac2 Docker Runner': 'mac2',
    'Linux Yoga Docker Runner': 'linux',
}

for name, r in results.items():
    counts = Counter(r['selections'])
    total = len(r['selections'])
    print(f'--- {name} ---')
    for runner in GROUND_TRUTH['docker-any']:
        n = counts.get(runner, 0)
        pct = n / total * 100
        bar = '▓' * int(pct / 2)
        short = runners_short.get(runner, runner[:10])
        print(f'  {short:8} {n:4}/{total}  ({pct:5.1f}%)  {bar}')
    print()

## 6. Results: Convergence Analysis

In [ ]:
def convergence_round(selections: list, optimal: str, window: int = 20, threshold: float = 0.7) -> int:
    """Find the round where strategy converges to selecting optimal runner."""
    for i in range(window, len(selections)):
        recent = selections[i-window:i]
        if recent.count(optimal) / window >= threshold:
            return i - window
    return -1  # didn't converge

# Optimal runner for docker-any is Linux Yoga (highest reward:cost ratio)
optimal = 'Linux Yoga Docker Runner'

print(f'=== Convergence to Optimal Runner ({runners_short[optimal]}) ===\n')
print(f'Criterion: ≥70% selection rate in rolling window of 20\n')

for name, r in results.items():
    conv = convergence_round(r['selections'], optimal)
    if conv >= 0:
        print(f'{name:15} converged at round {conv}')
    else:
        pct = Counter(r['selections']).get(optimal, 0) / len(r['selections']) * 100
        print(f'{name:15} did not converge ({pct:.0f}% overall)')

## 7. Regret Curve (ASCII)

In [ ]:
def ascii_plot(series_dict: dict, title: str, width: int = 70, height: int = 20):
    """Simple ASCII line chart."""
    all_vals = [v for vals in series_dict.values() for v in vals]
    y_min, y_max = min(all_vals), max(all_vals)
    if y_max == y_min:
        y_max = y_min + 1
    x_len = max(len(v) for v in series_dict.values())
    
    symbols = {'Rule-Based': '.', 'Pure MAB': '+', 'NSAI': '*'}
    
    print(f'\n{title}')
    print('─' * (width + 10))
    
    grid = [[' '] * width for _ in range(height)]
    
    for name, vals in series_dict.items():
        sym = symbols.get(name, '?')
        step = max(1, len(vals) // width)
        for xi in range(0, min(len(vals), width * step), step):
            col = xi // step
            if col >= width:
                break
            row = int((vals[xi] - y_min) / (y_max - y_min) * (height - 1))
            row = height - 1 - row  # invert y
            grid[row][col] = sym
    
    for i, row in enumerate(grid):
        y_val = y_max - (y_max - y_min) * i / (height - 1)
        print(f'{y_val:7.1f} │{".".join(row)}')
    
    print(f'        └{"─" * width}')
    print(f'         0{" " * (width-6)}round {x_len}')
    print(f'\n  Legend: ', end='')
    for name, sym in symbols.items():
        print(f'{sym}={name}  ', end='')
    print()

ascii_plot(
    {name: r['cum_regret'] for name, r in results.items()},
    'Cumulative Regret (lower is better)'
)

## 8. Live MAB Service Sync

In [ ]:
import urllib.request

MAB_URL = 'https://runner-bandit-m5cziijwqa-lz.a.run.app'

try:
    with urllib.request.urlopen(f'{MAB_URL}/stats', timeout=5) as resp:
        live_stats = json.loads(resp.read())
    
    print(f'MAB Service: {live_stats["algorithm"]}')
    print(f'Total observations: {live_stats["total_observations"]}\n')
    
    for runner, stats in live_stats.get('runners', {}).items():
        print(f'{runner:30} pulls={stats["pulls"]:3}  '
              f'success={stats["success_rate"]:.0%}  '
              f'avg_dur={stats["avg_duration"]:.1f}s  '
              f'reward={stats["mean_reward"]:.3f}')
    
    # Warm-start NSAI from live data
    nsai_live = NeurosymbolicBandit.from_live_service(MAB_URL)
    print(f'\n✅ NSAI warm-started from live service')
    print(f'   Total pulls synced: {nsai_live._total_pulls}')
    
except Exception as e:
    print(f'⚠️ MAB service not reachable: {e}')
    print('Continuing with cold-start NSAI.')

## 9. NSAI Explanation Demo

In [ ]:
# Create NSAI with some training
nsai = NeurosymbolicBandit.create_default()
rng = random.Random(42)

# Quick training
for _ in range(30):
    runner, _ = nsai.select_runner({'tags': ['docker-any']})
    success, dur = simulate_job(runner, 'docker-any', rng)
    nsai.update(runner, success, dur)

print('=== Docker Job Selection ===')
runner, exp = nsai.select_runner({'tags': ['docker-any']}, job_name='test:unit')
print(exp)

print('\n\n=== GCP-Only Job Selection ===')
runner, exp = nsai.select_runner({'tags': ['docker-any', 'gcp']}, job_name='cloud-run:build')
print(exp)

print('\n\n=== Impossible Job ===')
runner, exp = nsai.select_runner({'tags': ['gpu', 'arm64']}, job_name='ml:train')
print(exp)

## 10. Summary Statistics

In [ ]:
print('╔══════════════════════════════════════════════════════════╗')
print('║           NSAI Experiment Summary                       ║')
print('╠══════════════════════════════════════════════════════════╣')
print(f'║  NSAI Version:      {__version__:>10}                         ║')
print(f'║  Runners:           {len(create_blauweiss_ontology().runners):>10}                         ║')
print(f'║  Experiment Rounds: {300:>10}                         ║')
print('╠══════════════════════════════════════════════════════════╣')

for name in ['NSAI', 'Pure MAB', 'Rule-Based']:
    r = results[name]
    cum_r = r['cum_reward'][-1]
    cum_reg = r['cum_regret'][-1]
    print(f'║  {name:15} reward={cum_r:7.1f}  regret={cum_reg:7.1f}    ║')

print('╠══════════════════════════════════════════════════════════╣')

# NSAI advantage
nsai_r = results['NSAI']['cum_reward'][-1]
rule_r = results['Rule-Based']['cum_reward'][-1]
mab_r = results['Pure MAB']['cum_reward'][-1]
print(f'║  NSAI vs Rule-Based: {((nsai_r/rule_r - 1)*100):+.1f}% reward              ║')
print(f'║  NSAI vs Pure MAB:   {((nsai_r/mab_r - 1)*100):+.1f}% reward              ║')
print('╚══════════════════════════════════════════════════════════╝')